In [1]:
import os
import numpy as np
from sndaq.reader import SN_PayloadReader
from sndaq.writer import SN_PayloadWriter, construct_payload

#### Read one SN payload from existing file

In [2]:
data_dir = '../../data'
data_file = 'sn_209058_000001_239420_529095.dat'
with SN_PayloadReader(os.path.join(data_dir, data_file)) as r:
    pay = next(r)
print(pay)

Supernova@157144851073427008[dom 88ce18beda45 clk 03103a410001 scalerData*652


#### Custom Payload construction & Writing Payloads
Test `sndaq.writer.SN_PayloadWriter` and `sndaq.writer.construct_payloads` via the following steps
1. Write an existing SN payload to file
2. Construct a brand new custom SN payload
3. Write this custom payload to file
4. Read this file to check for equivalence

__Notes__
 - Times chosen below are arbitrary for sake of example, variable names containing `utime` reflect the time of an event in UTC, since the start of the year, measured in 0.1 ns
 - Scaler counts follow a poisson distribution, `scaler_lambda` was arbitrariy selected
 - Dictionary with keys matching keyword args of construct_payload may be unpacked via **
   - These values can just be passed as arguments to `construct_payload`, I just like using dicts for example code
 

In [3]:
test_file = '../../data/test.dat'
with SN_PayloadWriter(test_file, overwrite=True) as w:
    ### Test writer against existing payload
    w.write(pay)
    
    ### Test Construction of custom payload
    
    utime = 123456789  # Time of payload collection
    launch_utime = 120006539  # Time of DOM power-on
    n_scalers = 670
    scaler_lambda = 0.8  # 

    
    custom_pay_dict = {
        'utime': utime, 
        'dom_id': 0xe9fed8c717dd,  # DOM [1-6] Chutes_and_Ladders, value equivalent to 257280767891421
        'domclock': (utime - launch_utime)//250,  # Integer number of 25 ns clock cycles since DOM activation
        'scalers': np.random.poisson(scaler_lambda, size=n_scalers),
        'keep_data': True
    }
    
    custom_payload = construct_payload(**custom_pay_dict)
    
    ### Test writer against custom payload
    w.write(custom_payload)

In [4]:
from sndaq.detector import Detector
i3 = Detector()
i3.dom_table

array([( 1.,  1., -256.14, -521.08,  496.07,  47303335284587, 'i3', 0.140612),
       ( 1.,  2., -256.14, -521.08,  479.05,  20579555797555, 'i3', 0.151956),
       ( 1.,  3., -256.14, -521.08,  462.03, 106634453247646, 'i3', 0.168092),
       ...,
       (86., 58.,  -10.97,    6.72, -486.5 , 255380957221937, 'dc', 0.29648 ),
       (86., 59.,  -10.97,    6.72, -493.51, 107432791511293, 'dc', 0.291581),
       (86., 60.,  -10.97,    6.72, -500.83, 280205879548048, 'dc', 0.285821)],
      dtype=[('str', '<f4'), ('i', '<f4'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('mbid', '<u8'), ('type', '<U2'), ('effvol', '<f8')])

In [5]:
pay.data_bytes

b'\x00\x00\x88\xce\x18\xbe\xdaE\x02\x96\x01,\x03\x10:A\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x01\x00\x00\x00\x00\x01\x00\x02\x01\x00\x00\x02\x01\x01\x00\x00\x01\x01\x00\x00\x00\x00\x00\x01\x00\x01\x00\x01\x01\x01\x01\x00\x00\x00\x02\x01\x00\x00\x01\x00\x01\x00\x01\x01\x01\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x02\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x03\x00\x02\x01\x01\x01\x00\x00\x00\x00\x01\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x02\x00\x00\x00\x00\x01\x01\x01\x00\x01\x01\x00\x00\x01\x00\x00\x00\x00\x01\x01\x01\x03\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x01\x00\x00\x00\x00\x01\x02\x00\x00\x01\x00\x01\x00\x00\x00\x00\x00\x01\x01\x01\x00\x00\x00\x00\x00\x02\x01\x00\x01\x00\x01\x01\x01\x00\x00\x00\x00\x00\x02\x00\x00\x01\x00\x01\x01\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x02\x00\x04\x01\x00\x01\x02\x01\x00\x00\x01\x

In [6]:
test_file = '../../data/test.dat'
with SN_PayloadWriter(test_file, overwrite=True) as w:
    ### Test writer against existing payload
    w.write(pay)
    
    ### Test Construction of custom payload
    
    utime = 238749318  # Time of payload collection
    launch_utime = 120006539  # Time of DOM power-on
    n_scalers = 650
    scaler_lambda = 0.8  # 

    
    custom_pay_dict = {
        'utime': utime, 
        'dom_id': 0xe9fed8c717dd,  # DOM [1-6] Chutes_and_Ladders, value equivalent to 257280767891421
        'domclock': (utime - launch_utime)//250,  # Integer number of 25 ns clock cycles since DOM activation
        'scalers': np.random.poisson(scaler_lambda, size=n_scalers),
        'keep_data': True
    }
    
    custom_payload = construct_payload(**custom_pay_dict)
    
    ### Test writer against custom payload
    w.write(custom_payload)

#### Sanity check via print(payload) before reading from file

In [7]:
print('=== Expected Custom Payload __str__ (Sanity check only, Not a test of equivalence) ===')
print("Supernova@{0:d}[dom {1:012x} clk {2:012x} scalerData*{3:d}]\n".format(
    utime, custom_pay_dict['dom_id'], custom_pay_dict['domclock'], n_scalers))
print(custom_payload)

=== Expected Custom Payload __str__ (Sanity check only, Not a test of equivalence) ===
Supernova@238749318[dom e9fed8c717dd clk 000000073f5b scalerData*650]

Supernova@238749318[dom e9fed8c717dd clk 000000073f5b scalerData*650


#### Sanity check via print(payload) after reading from file

In [8]:
with SN_PayloadReader(test_file) as r:
    while r.nrec < 2:
        pay = next(r)
        print(pay)
        
print('\n=== Expected Custom Payload __str__ (Sanity check only, Not a test of equivalence) ===')
print("Supernova@{0:d}[dom {1:012x} clk {2:012x} scalerData*{3:d}]".format(
    utime, custom_pay_dict['dom_id'], custom_pay_dict['domclock'], n_scalers))

Supernova@157144851073427008[dom 88ce18beda45 clk 03103a410001 scalerData*652
Supernova@238749318[dom e9fed8c717dd clk 000000073f5b scalerData*650

=== Expected Custom Payload __str__ (Sanity check only, Not a test of equivalence) ===
Supernova@238749318[dom e9fed8c717dd clk 000000073f5b scalerData*650]


## Creating sndata for Unit-testing

Real sndata.dat files are constructed taking into consideration a few principles
1. Every payload in a .dat file is arranged in ascending time, and every .dat file is also ordered in ascending times
2. Every sequential payload for a particular DOM has a utime that corresponds to the scaler readout immediately following the last scaler in the previous file.
 - Scalers are collected over a period of $2^{16}$ clock cycles, so in units 0.1 ns, that is $2^{16} \times 250 = 16384000$, or 1.6384 ms.
 - In brief, each scaler is collected 1.6384 ms after the previous scaler.
 - The payload `utime` and `domclock` fields refers to the time and no. of DOM clock cycles at the first scaler in the payload
 - For DOM $i$ with first payload $p_{i,0}$, `utime` $t_{i,0}$ and $n_{i,0}$ scalers, the second payload for that DOM in the file $p_{i,1}$ will have `utime` $t_{i,1} = t_{i,0} + n_{i,0} \times (2^{16} \times 250)$
3. Real sndata files collect ~190 MB of data before a new file is created
4. Every DOM in the run configuration issues payload $n$ before any DOM issues paylaod $n+1$

Committing large files is not recommended, but having some files would be useful for unit testing.

A data set would look something like the following

 - Small size (<25MB). 
   - File size is prop. to duration and number of DOMs. Having a longer duration is more important for testing, so a test set might only need one or several strings of the detector. 
 - Adheres to the file format/principles above
 - Contains specific features (Unless it is intended to represent background) that can affect trigger significance, to test trigger formation algorithm
   - Occasionally in real data, the one or more of the conditions will not be met, and SNDAQ will handle it somewhat quietly (usually) For the sake of testing, we should assume all of these conditions to be met, unless we are making a test set to specifically examine how SNDAQ handles these circumstances.